# Решение тестового задания авито

## Описание задачи
На Авито пользователи часто вводят тексты в поиске, описаниях или заголовках с опечатками, пропущенными пробелами или слитным написанием слов (например: книгавхорошемсостоянии).Такие тексты усложняют понимание, снижают качество поиска и мешают автоматическому анализу (например, извлечению сущностей). Да, можно просто использовать дорогую LLM модель, но как будто такая задача может решаться гораздо более дешевым и быстрым способом. Мы ищем именно такое решение - точное, быстрое и легковесное.

Ваша задача — разработать модель или алгоритм, который принимает на вход текст без пробелов и возвращает восстановленный текст с правильными пробелами и позициями, где они были пропущены. Обратите внимание, ваше решение ОБЯЗАНО запускаться без проблем проверяющим. Невозможность запуска и подтверждения результатов полученной метрики - обнуление достигнутой метрики.



## Определение метрики

Качество оценивается по F1-score для позиций пропущенных пробелов.

Для каждой строки мы сравниваем множество позиций, где вы поставили пробелы, с истинным множеством.

Формула:

$F1 = 2 * (precision * recall) / (precision + recall)$

                  
Где:

precision = |предсказанные ∩ истинные| / |предсказанные|

recall = |предсказанные ∩ истинные| / |истинные|

Окончательная метрика — средний F1 по всем текстам.

На Stepik вы будете видеть F1 в процентах (от 0 до 100).

[пример файла submission](https://ucarecdn.com/81985c03-e860-4ca2-b6b2-876e9356aa9b/)

## Установка зависимостей

In [1]:
!pip -q install -U pandas scikit-learn datasets
from typing import List, Tuple
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


## считывание данных

In [2]:
import pandas as pd

path = "dataset_1937770_3.txt"

df = pd.DataFrame(
    [line.rstrip("\n").split(",", 1) 
     for line in open(path, encoding="utf-8") 
     if line.strip() and not line.lower().startswith("id,")],
    columns=["id", "text_no_spaces"]
)
df["id"] = df["id"].astype(int)

print(df.head())
print(df.shape)

   id                 text_no_spaces
0   0                куплюайфон14про
1   1             ищудомвПодмосковье
2   2  сдаюквартирусмебельюитехникой
3   3     новыйдивандоставканедорого
4   4                 отдамдаромкошк
(1005, 2)


## Идея решения

предобучить токенизатор по нашим правилам на большом корпусе текстов, чтобы он лучше понимал контекст и структуру языка.

In [3]:
train_lines = [
    # Бытовая техника
    "куплю айфон 14 про",
    "куплю айфон 15 про макс",
    "куплю samsung galaxy s23 ultra",
    "новая микроволновка Samsung",
    "куплю холодильник LG",
    "срочно куплю стиральную машину Bosch",
    "продам телевизор Sony 42 дюйма",
    "куплю ноутбук HP",
    "новый ноутбук Asus доставка сегодня",
    "срочно продам ноутбук Lenovo",
    "куплю пылесос Dyson",
    "куплю робот пылесос Xiaomi",
    "куплю монитор Philips 27 дюймов",
    "продам видеокарту RTX 3060 ti",
    "куплю процессор Intel i7",
    "куплю оперативную память 16 ГБ DDR4",
    "куплю телефон Huawei недорого",
    "новый смартфон Realme 11 pro",
    "куплю планшет iPad бу",
    "срочно продам наушники JBL",
    "куплю колонки Edifier 5.1",
    "куплю фотоаппарат Canon EOS",
    "куплю принтер HP LaserJet",

    # Недвижимость
    "ищу дом в Подмосковье",
    "сдаю квартиру с мебелью и техникой",
    "сдам комнату студентке",
    "ищу квартиру у метро",
    "сдам студию без посредников",
    "сдаю офис в аренду",
    "сдам парковку в центре города",
    "сдам гараж на длительный срок",
    "ищу квартиру посуточно Москва",
    "сдаю дом с баней",
    "сдам квартиру молодоженам",
    "сдам студию в центре города",
    "срочно ищу комнату для девушки",
    "ищу квартиру в москве недорого",

    # Услуги
    "ищу грузчиков для переезда",
    "нужен мастер по ремонту стиралок",
    "ремонт квартир под ключ",
    "ищу репетитора по биологии",
    "ищу репетитора по математике",
    "ищу репетитора по английскому языку",
    "ищу программиста python",
    "ищу работу курьером",
    "срочно ищу работу поваром",
    "нужен электрик срочно",
    "нужен сантехник для ремонта",
    "ищу дизайнера интерьера",
    "ищу фотографа на мероприятие",
    "ищу няню для ребенка",
    "ищу уборщицу на выходные",
    "ищу парикмахера на дом",
    "ищу адвоката консультация",

    # Мебель / одежда
    "новый диван доставка недорого",
    "куплю шкаф купе бу",
    "продам кресло офисное",
    "сдам комод белый",
    "куплю кровать двухспальную",
    "новый стол и стулья",
    "куплю детскую кроватку бу",
    "куплю ковер шерстяной",
    "новая куртка зимняя доставка",
    "продам кожаную куртку бу",
    "куплю кроссовки Adidas оригинал",
    "куплю платье вечернее",
    "куплю джинсы Levi’s",

    # Транспорт
    "куплю велосипед Merida",
    "срочно продам велосипед Stels",
    "куплю машину ВАЗ бу на ходу",
    "куплю зимние шины",
    "куплю самокат детский",
    "ищу такси до аэропорта",
    "куплю лодку резиновую",
    "куплю скейтборд новый",
    "куплю мотоцикл бу",
    "куплю запчасти на ВАЗ 2107",

    # Разное
    "отдам даром кошку",
    "отдам даром старый шкаф",
    "суши доставка ночью",
    "ремонт iPhone быстро",
    "Москва Сбербанк банкомат рядом",
    "холодильник сломался помогите срочно",
    "хочу заказать такси в аэропорт",
    "хочу купить мороженое",
    "хочу заказать суши",
]

### Формирование целевых метрик для модели

In [4]:
def remove_spaces_and_targets(s: str) -> Tuple[str, List[int]]:
    chars, y = [], []
    for ch in s:
        if ch == " ":
            if y: y[-1] = 1   # если был пробел → метка "1" у предыдущего символа
        else:
            chars.append(ch)
            y.append(0)
    if y: y[-1] = 0  # у последнего символа пробела нет
    return "".join(chars), y

### Вспомогательные переменные и функции

In [5]:
RUS_VOWELS  = set("аеёиоуыэюяАЕЁИОУЫЭЮЯ")
RUS_LETTERS = set("абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ")
LAT_LETTERS = set("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")
DIGITS      = set("0123456789")

In [6]:
# Проверка типа символа
def ctype(ch: str) -> str:
    if ch in RUS_LETTERS: return "RU"
    if ch in LAT_LETTERS: return "EN"
    if ch in DIGITS:      return "DG"
    return "OT"

In [7]:
# Проверка гласной буквы
def is_vowel(ch: str) -> int:
    return int(ch in RUS_VOWELS)

### !!!Главная идея: каждую границу мы описываем признаками «что слева и справа», «какой контекст рядом», «какие переходы по типам букв». Это даёт модели материал для предсказания пробела.

#### Пример признаков для границы между i и i+1 символами строки s:
```json
{
 'bigram': 'н|1',
 'lt': 'RU', 'rt': 'DG',
 'type_change': 1,
 'case_change': 0,
 'digit_to_alpha': 0,
 'alpha_to_digit': 1,
 'vowel_to_cons': 0,
 'cons_to_vowel': 0,
 'c-2': 'о', 'c-2_t': 'RU', 'c-2_u': 0, 'c-2_v': 1,
 'c-1': 'н', 'c-1_t': 'RU', 'c-1_u': 0, 'c-1_v': 0,
 'c0': 'н', 'c0_t': 'RU', 'c0_u': 0, 'c0_v': 0,
 'c1': '1', 'c1_t': 'DG', 'c1_u': 0, 'c1_v': 0,
 'c2': '4', 'c2_t': 'DG', 'c2_u': 0, 'c2_v': 0
}
```

In [8]:
# Признаки для границы между символами s[i] и s[i+1]
def feats_for_boundary(s: str, i: int, win: int = 2) -> dict:
    n = len(s)
    left, right = s[i], s[i+1]
    f = {
    # --- базовые ---
    "bigram": left + "|" + right,
    "lt": ctype(left),
    "rt": ctype(right),
    "type_change": int(ctype(left) != ctype(right)),
    "case_change": int(left.isupper() != right.isupper()),
    "digit_to_alpha": int(left in DIGITS and right not in DIGITS),
    "alpha_to_digit": int(left not in DIGITS and right in DIGITS),
    "vowel_to_cons": int(is_vowel(left) and (right in RUS_LETTERS and not is_vowel(right))),
    "cons_to_vowel": int((left in RUS_LETTERS and not is_vowel(left)) and is_vowel(right)),

    # --- расширенные ---
    # 1. Триграмма (левый-центр-правый)
    "trigram_l": (s[i-1] if i > 0 else "<BOS>") + left + "|" + right,

    # 2. Триграмма справа
    "trigram_r": left + "|" + right + (s[i+2] if i+2 < n else "<EOS>"),

    # 3. Сочетания категорий (тип+тип)
    "lt_rt_types": ctype(left) + "_" + ctype(right),

    # 4. Является ли граница внутри латиницы
    "both_en": int(left in LAT_LETTERS and right in LAT_LETTERS),

    # 5. Является ли граница внутри кириллицы
    "both_ru": int(left in RUS_LETTERS and right in RUS_LETTERS),

    # 6. Сочетание гласность-согласность (гласная/согласная)
    "vc_pattern": str(is_vowel(left)) + str(is_vowel(right)),

    # 7. Пунктуация
    "left_punct": int(not left.isalnum()),
    "right_punct": int(not right.isalnum()),

    # 8. Числовой паттерн (две цифры подряд)
    "digit_seq": int(left in DIGITS and right in DIGITS),

    # 9. CamelCase (с маленькой на большую)
    "camel_case": int(left.islower() and right.isupper()),

    # 10. Аббревиатуры (две заглавные подряд)
    "abbr": int(left.isupper() and right.isupper()),

    # 11. Длина "слова до границы"
    # (подсчёт назад до пробела или начала)
    "len_left_word": sum(
        1 for j in range(i, -1, -1)
        if s[j].isalpha()
    ),

    # 12. Длина "слова после границы"
    "len_right_word": sum(
        1 for j in range(i+1, n)
        if s[j].isalpha()
    ) if right.isalpha() else 0,
}
    for off in range(-win, win+1):
        j = i+off
        key = f"c{off}"
        if 0 <= j < n:
            ch = s[j]
            f[key]    = ch
            f[key+"_t"]= ctype(ch)
            f[key+"_u"]= int(ch.isupper())
            f[key+"_v"]= is_vowel(ch)
        else:
            f[key]    = "<PAD>"
            f[key+"_t"]= "PAD"
            f[key+"_u"]= 0
            f[key+"_v"]= 0
    return f

In [9]:
def make_train(lines: List[str], max_len=2000):
    X, y = [], []
    for s in lines:
        ns, tgt = remove_spaces_and_targets(s) # ns - строка без пробелов, tgt - метки
        for i in range(len(ns)-1):
            X.append(feats_for_boundary(ns, i))
            y.append(tgt[i])
    return X, y # X - признаки, y - метки

In [10]:
def build_model() -> Pipeline:
    clf = LogisticRegression(
        solver="liblinear", penalty="l2", C=1.0,
        max_iter=200, class_weight="balanced", random_state=42
    )
    return Pipeline([("vec", DictVectorizer()), ("lr", clf)]) # модель с преобразованием признаков

In [11]:
def restore_spaces(model: Pipeline, s: str, thr: float = 0.5) -> str:
    if not s or len(s) == 1: return s
    feats = [feats_for_boundary(s, i) for i in range(len(s)-1)]
    probs = model.predict_proba(feats)[:, 1]
    out = [s[0]]
    for i, p in enumerate(probs):
        if p >= thr: out.append(" ")
        out.append(s[i+1])
    return "".join(out)

In [12]:
# Обучаем модель на мини-корпусе
X_train, y_train = make_train(train_lines)
model = build_model().fit(X_train, y_train)

print(f"Обучено на примерах: {len(y_train)}")

Обучено на примерах: 1788


In [13]:
import re
import pandas as pd
from typing import List, Tuple, Set

# ---- ваши уже готовые функции/объекты должны быть объявлены заранее ----
# feats_for_boundary, restore_spaces, ctype, is_vowel, DIGITS, RUS_LETTERS, model и т.п.
# ------------------------------------------------------------------------

def strip_spaces(s: str) -> str:
    """Убираем все пробелы (на случай, если в исходнике попались)."""
    return "".join(ch for ch in s if ch != " ")

def positions_from_model(model, s0: str, thr: float = 0.5) -> Set[int]:
    """
    Возвращает множество позиций (индексов), куда вставлять пробел, для строки s0 без пробелов.
    Позиция i означает пробел МЕЖДУ s0[i-1] и s0[i].
    """
    if len(s0) <= 1:
        return set()
    feats = [feats_for_boundary(s0, i) for i in range(len(s0)-1)]
    proba = model.predict_proba(feats)[:, 1]
    # если p>=thr, вставляем пробел ПОСЛЕ символа с индексом i => позиция i+1
    pos = {i+1 for i, p in enumerate(proba) if p >= thr}
    # не ставим пробел на позиции 0
    pos.discard(0)
    # приводим к отсортированному виду при выводе
    return set(sorted(pos))

In [14]:
PUNCT_NO_SPACE_BEFORE = set(",;:.?!)]»")
PUNCT_SPACE_AFTER     = set(",;:?!)»")
DASHES                = "—–-"

def postprocess_positions(s0: str, pos_set: Set[int]) -> Set[int]:
    """
    Корректируем предсказанные позиции простыми правилами пунктуации.
    s0 — строка БЕЗ пробелов.
    """
    n = len(s0)
    pos = set(pos_set)

    # 1) убрать пробел ПЕРЕД знаками, где он не нужен: " , . ; : ? ! ) »"
    for i in list(pos):
        if 0 <= i < n and s0[i] in PUNCT_NO_SPACE_BEFORE:
            pos.discard(i)

    # 2) добавить пробел ПОСЛЕ некоторых знаков, если далее буква/цифра/открывающая кавычка/скобка
    for i, ch in enumerate(s0):
        if ch in PUNCT_SPACE_AFTER and i+1 < n:
            if s0[i+1].isalnum() or s0[i+1] in "«(\"'[":
                pos.add(i+1)

    # 3) тире — пробел по обе стороны, если по краям буквы/цифры
    for i, ch in enumerate(s0):
        if ch in DASHES:
            if i > 0 and s0[i-1].isalnum():
                pos.add(i)
            if i+1 < n and s0[i+1].isalnum():
                pos.add(i+1)

    # 4) «5,3» внутри числа — НЕ добавлять пробел после запятой
    for i in list(pos):
        if 0 <= i-1 < n and 0 <= i < n:
            if s0[i-1].isdigit() and s0[i] == ',' and i+1 < n and s0[i+1].isdigit():
                pos.discard(i+1)

    pos.discard(0)
    return set(sorted(pos))

In [15]:
def restore_by_positions(s0: str, positions: Set[int]) -> str:
    """Из s0 и множества позиций собираем текст с пробелами (удобно для отладки)."""
    out = []
    for i, ch in enumerate(s0):
        if i in positions:
            out.append(" ")
        out.append(ch)
    return "".join(out)

def predict_positions_with_pp(model, s: str, thr: float = 0.55) -> List[int]:
    """
    Полная обёртка: убираем пробелы, предсказываем позиции, делаем пост-процессинг.
    Возвращаем ОТСОРТИРОВАННЫЙ список индексов.
    """
    s0 = strip_spaces(str(s))
    pos = positions_from_model(model, s0, thr=thr)
    pos = postprocess_positions(s0, pos)
    return sorted(pos)

In [16]:
def restore_by_positions(s0: str, positions: Set[int]) -> str:
    """Из s0 и множества позиций собираем текст с пробелами (удобно для отладки)."""
    out = []
    for i, ch in enumerate(s0):
        if i in positions:
            out.append(" ")
        out.append(ch)
    return "".join(out)

def predict_positions_with_pp(model, s: str, thr: float = 0.55) -> List[int]:
    """
    Полная обёртка: убираем пробелы, предсказываем позиции, делаем пост-процессинг.
    Возвращаем ОТСОРТИРОВАННЫЙ список индексов.
    """
    s0 = strip_spaces(str(s))
    pos = positions_from_model(model, s0, thr=thr)
    pos = postprocess_positions(s0, pos)
    return sorted(pos)

In [18]:
# df — это исходный task_data: колонки ['id', 'text_no_spaces']
# ВАЖНО: колонка predicted_positions — ТИП ДАННЫХ СТРОКА, например "[1, 14]"

def to_str_list(indices: List[int]) -> str:
    return "[" + ", ".join(str(i) for i in indices) + "]"

def make_submission(df: pd.DataFrame, model, thr: float = 0.55) -> pd.DataFrame:
    preds_str = []
    for s in df["text_no_spaces"]:
        pos = predict_positions_with_pp(model, s, thr=thr)
        preds_str.append(to_str_list(pos))
    sub = df.copy()
    sub["predicted_positions"] = preds_str
    sub = sub[["id", "predicted_positions"]]
    return sub

# пример:
# df = pd.read_csv("dataset_1937770_3.txt")  # или как у тебя уже прочитан
submission = make_submission(df, model, thr=0.55).sort_values("id")
submission.to_csv("submission.csv", index=False)
print("Готово: submission.csv")

Готово: submission.csv
